In [1]:
!pip install openai -q


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai
print(openai.__version__)

1.54.3


In [4]:
pip install --upgrade openai


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import the OpenAI library to interact with the OpenAI API.
from openai import OpenAI
import textwrap
import requests  # used to download images
import os  # used to access filepaths
from PIL import Image  # used to print and edit images
import base64
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import pandas as pd  # To read and handle the CSV file
import openai  # To use GPT-4 for evaluation
import logging  # For logging results and errors
from PIL import Image  # To handle images (Pillow)
import logging
import csv
import json



api_key = "sk-svcacct-vdeZ2LdFD4WLSoWokO9JpnArMctwcXj1PFex5qsr-zPmffZrs9yG6AbZI4eJHSYbdST3BlbkFJkyirN41YuhOaTHPV2o-VPmd4Ydj_K28bTn6gvV2MfIci_04uGe2tprRE-yWbRxoWUA"
client = OpenAI(api_key=api_key)
#judgeGPT()

In [6]:
def load_smmu_tasks(csv_file_path, images_folder_path):
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Ensure image folder path exists
    if not os.path.exists(images_folder_path):
        raise FileNotFoundError(f"The images folder '{images_folder_path}' does not exist.")
    
    # Add the image file path to each task by constructing it from the Image_ID column
    def add_image_path(row):
        image_filename = row['Image_ID']  # Image filename should already include the extension, e.g., 'image_1.jpg'
        image_path = os.path.join(images_folder_path, image_filename)
        return image_path
    
    # Apply the function to add the image path to the DataFrame
    df['image_path'] = df.apply(add_image_path, axis=1)
    df['task_id'] = [f"task_{i+1}" for i in range(len(df))]
    
    # Convert to list of dictionaries for easy use in your loop
    tasks = df.to_dict(orient="records")
    return tasks


In [7]:
csv_file_path = "./ocr_QA.csv"
images_folder_path = "./ocr_images"

tasks = load_smmu_tasks(csv_file_path, images_folder_path)

# Print each task in an organized way
for i, task in enumerate(tasks, start=1):
    print(f"{task['task_id']}:")
    print(f"  Image Path  : {task['image_path']}")
    print(f"  Question    : {task['question']}")
    print(f"  Answer      : {task['answer']}")
    print("-" * 40)  # Divider between tasks for clarity


task_1:
  Image Path  : ./ocr_images/image_1.jpg
  Question    : What is the level of the Chinese language textbook depicted in the image?
  Answer      : The level of the Chinese language textbook depicted in the image is Level 3, as indicated by the number "3" on the bottom right-hand side of the cover.
----------------------------------------
task_2:
  Image Path  : ./ocr_images/image_1.jpg
  Question    : How many people are shown on the textbook cover?
  Answer      : There are five people shown on the textbook cover, with three individuals prominently in the foreground and two in the background.
----------------------------------------
task_3:
  Image Path  : ./ocr_images/image_1.jpg
  Question    : What is the predominant color of the textbook's logo at the bottom of the image?
  Answer      : The predominant color of the textbook's logo at the bottom of the image is blue.
----------------------------------------
task_4:
  Image Path  : ./ocr_images/image_1.jpg
  Question    : W

In [8]:
def generate_model_answer(task):
    """
    Generate an answer for the given task using a model. This is a placeholder function.
    Replace this code with actual model inference logic to produce meaningful answers.

    Parameters:
    task (dict): A dictionary containing information for a specific task, such as the question and options.

    Returns:
    str: The answer generated by the model.
    """
    
    # Placeholder logic for model inference
    # Replace the following line with actual inference code for your model
    # For example, use a model prediction based on `question` and `options`
    model_answer = "Simulated model answer for testing."  # Replace this line
    
    return model_answer


In [9]:
EVALUATION_SYSTEM_PROMPT = """
You are an expert evaluator specializing in assessing questions in primary school to high school exams. I will give you a question, the expected correct answer, and a test-taker's response to the question.
You need to analyse the given question,compare the standard answer with the provided response, and fill in the following values:
- analysis: If the answer is incorrect, you need to give a reason for the error. If the answer is correct, you can leave it blank. The analysis must be a string, not exceeding 500 characters.
- correct: Whether the answer to the question is correct and have necessary info. Return 1 for correct, 0 for incorrect.
The above values should be returned in JSON format. I should be able to directly load the return value into a dict variable using the json.loads function in Python.

Remember, your output should only contain the following format:
{
"analysis":,
"correct":
}
Be sure to use double backslashes if necessary, not single backslashe. 
"""

EVALUATION_USER_TEMPLATE = """
Here is the question:
"{}"

The expected correct answer to this problem:
"{}"

Response to the problem:
"{}"
"""

In [10]:
import openai

def gpt4_judge(reference_answer, model_answer, question):
    # Construct the prompt for the model
    prompt = (EVALUATION_USER_TEMPLATE.format(reference_answer, model_answer, question))
    
    # Make sure you're using the chat completions endpoint for GPT-4
    response = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": EVALUATION_SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=640,
        top_p=1
    )
    judgement = response.choices[0].message.content.strip()
    
    try:
        judgement_json = json.loads(judgement)
        correct_value = judgement_json.get("correct", 0)
        analysis = judgement_json.get("analysis", "No analysis available")
        return correct_value, analysis

    except json.JSONDecodeError as e:
        print(f"Error parsing summary as JSON: {e}")
        return 0, "Error in analysis"


In [11]:
# Set up logging
logging.basicConfig(filename="evaluation.log", level=logging.INFO)

def evaluate_smmu_task(task_id, task, repetitions=3):
    results = []
    correct_count = 0  # Track the number of correct responses
    total_count = repetitions
    
    for i in range(repetitions):
        # Generate the model's response
        model_answer = generate_model_answer(task)  # Replace with actual model function
        
        # Use the GPT-4 model to judge the answer
        judgement,analysis = gpt4_judge(task["answer"], model_answer, task["question"])
        
        # Calculate correctness
        correct = 1 if judgement else 0
        correct_count += correct

        # Save the results to CSV
        save_results_to_csv(task["task_id"], task["question"], task["answer"], model_answer, correct, analysis, output_csv)

        # Log each judgement result
        logging.info(f"Task {task_id}, Run {i}, Judgement: {judgement}, Analysis: {analysis}")

        # Store results for further use
        results.append({"run": i, "model_answer": model_answer, "judgement": judgement, "analysis": analysis})
       
    # Calculate accuracy and log it
    accuracy = correct_count / total_count
    logging.info(f"Task {task_id} Accuracy: {accuracy * 100:.2f}%")
    
    return results


In [12]:
# Prepare the CSV file with headers
output_csv = "evaluation_results.csv"
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["task_id", "question", "answer", "response", "correctness", "analysis"])

def save_results_to_csv(task_id, question, answer, response, correctness, analysis, output_csv):
    with open(output_csv, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([task_id, question, answer, response, correctness, analysis])

In [ ]:
def main():
    csv_file_path = "./ocr_QA.csv"
    images_folder_path = "./ocr_images"
    
    # Load tasks
    tasks = load_smmu_tasks(csv_file_path, images_folder_path)
    
    # Dictionary to hold all results
    all_results = {}
    
    # Loop over tasks and evaluate
    for task_id, task in enumerate(tasks):  # Corrected loop
        task_results = evaluate_smmu_task(task_id, task, repetitions=3)  # Example with 3 repetitions
        all_results[task_id] = task_results
    print("Evaluation complete. Results saved to results.json and results.csv.")

if __name__ == "__main__":
    main()

In [ ]:
# Example analysis to get success rate per task
results_df = pd.read_csv("evaluation_results.csv")
success_rate = results_df.groupby("task_id")["correctness"].apply(lambda x: sum(x == "1") / len(x))
print(success_rate)

In [ ]:
print(task)